<img src="logo.png">

# Familia apply

Ya hemos visto que la gran mayoría de operaciones en **R** son vectorizadas. 

Los lenguajes de programación con operaciones vectorizadas ofrecen una gran capacidad para realizar tareas en menos tiempo que utilizado un ciclos for o while.

Por ejemplo, considere dos vectores ``x`` y ``y`` de la misma longitud. Supongamos que queremos obtener un vector ``z`` formado por la suma de los elementos de ``x`` con los elementos de ``y`` de manera ordenada.

``
n = length(x)
for(i in 1:n){
   z[i] <- x[i]+y[i]
}
``

Si $n$ es muy grande, la instrucción anterior podría tardar **muchísimo**, en tanto que la ventaja de la vectorización de la suma en **R** es que, aun para $n$ grande, ``z=x+y`` es relativamente más rápida que el uso de un ciclo.

Por su parte, **R** cuenta con una serie de funciones, la **familia apply**, tal que cada uno de sus elementos es una función vectorizada diseñada para sustituir ciclos for y while de manera eficiente para aplicar otras funciones.

Esta familia está formada por ``lapply()``, ``tapply()``, ``sapply()``, ``mapply()`` y ``rapply()``

En este capítulo estudiaremos las mas importantes.

## tapply( )

Esta función es diferente a las anteriores. Sirve para aplicar una función a cierta columna de un dataframe categorizando por otra(s) columna(s) del mismo dataframe:

``tapply(col,list(col_cat1,col_cat2,...),FUN)``

donde

- ``col`` es la columna a la que se le aplicará la función ``FUN``
- ``col_cat1``, ``col_cat2``,... son las columnas por las que se hará la segregación

In [ ]:
# calcular el mpg promedio por marca

tapply(mtcars$mpg,mtcars$marca,mean)

In [ ]:
# calcular el mínimo de disp por cilindros y engranaje

tapply(mtcars$disp,list(mtcars$cyl,mtcars$gear),min)

In [ ]:
### # Calcular la media, mediana y total de ventas por región y producto

ventas_data <- data.frame(
  Region = c("Norte", "Norte", "Sur", "Sur", "Este", "Este", "Oeste", "Oeste", "Norte", "Sur"),
  Producto = c("A", "B", "A", "B", "A", "B", "A", "B", "A", "A"),
  Ventas = c(150, 200, 250, 300, 100, 120, 80, 90, 200, 220)
)

tapply(ventas_data$Ventas,list(ventas_data$Region,ventas_data$Producto),function(x) {
  return(c(media = mean(x), mediana = median(x), total = sum(x)))
})


## lapply( )

La primer función a estudiar será ``lapply( )``. Su sintaxis es ``lapply(X,FUN)``. Aquí, ``X`` es un vector, una lista o un array, y ``FUN`` es una función que se aplicará a cada elemento de ``X``. El resultado será una lista con el mismo número de elementos de ``X``; se recomienda que ``X`` sea un vector o una lista.

In [ ]:
## Construir una lista de textos: textos = list("Texto 1","Texto 2",...,"Texto n")

mi_funcion1 <- function(x){
    paste0("Texto_",x)
}

lapply(1:100,mi_funcion1)

In [ ]:
#Dada una lista de textos, convertir las vocales a mayúsculas

cambio_vocales <- function(texto){
  n = nchar(texto)
  letras <- c()
  for(x in 1:n){
    letras[x] <- substr(texto,x,x)
    if(letras[x] %in% c("a","e","i","o","u")){
      letras[x] <- c("A","E","I","O","U")[letras[x]==c("a","e","i","o","u")]
    }
  }
  return(paste(letras,collapse=""))  
}


mis_frases <- c("La vida es una obra teatral que no importa cuánto haya durado, sino lo bien que haya sido representada",
                "Elige un trabajo que ames, y no tendrás que trabajar un solo día de tu vida",
                "Nuestra mayor gloria no es no caer nunca, sino levantarnos cada vez que caemos")

In [ ]:
# Lectura de archivos:

archivos <- c(1,2,4)
mis_archivos <- lapply(dir()[archivos],read.csv)
names(mis_archivos) <- unlist(strsplit(dir(),split=".csv"))[archivos]



In [ ]:
data(mtcars)

In [ ]:
mtcars

In [ ]:
# calcular el máximo y el promedio de cada de las columnas mpg, hp y wt

mi_max_prom <- function(columna){
    valores  <- c(max(columna),mean(columna))
    names(valores) = c("Maximo","Promedio")
    return(valores)
}

lapply(mtcars[,c("mpg","hp","wt")],mi_max_prom)


## sapply( )

La función ``sapply( )`` es muy parecida a ``lapply( )``, con la diferencia de que devuelve un vector, una matriz o un array:

``sapply(x,FUN,simplify = TRUE,USE.NAMES=TRUE)``

donde 

- ``simplify = TRUE`` devuelve una lista o una matriz. Si ``="array"`` devuelve un array.
- ``USE.NAMES = TRUE`` el resultado de aplicar ``FUN`` a cada elemento de ``x`` tendrá el mismo nombre.

In [ ]:
sapply(mis_frases,cambio_vocales,simplify = TRUE,USE.NAMES = TRUE)

In [ ]:
sapply(mtcars[,c("mpg","hp","wt")],mi_max_prom,simplify=TRUE, USE.NAMES=FALSE)

In [ ]:
## Insertar una columna a mtcars con la marca del automovil

nombres_partidos <- strsplit(row.names(mtcars)," ")
mtcars$marca <- sapply(nombres_partidos,function(x){x[1]})


In [ ]:
mtcars

In [ ]:
data <- data.frame(
  Altura = c(170, 165, 180, 175, 160),
  Peso = c(70, 65, 80, 75, 60),
  Edad = c(30, 25, 35, 28, 22)
)

# Función para calcular estadísticas descriptivas
calcular_estadisticas <- function(x) {
  c(media = mean(x), desviacion_std = sd(x), mediana = median(x))
}

# Aplicar la función a cada columna del DataFrame
estadisticas <- sapply(data, calcular_estadisticas)
estadisticas

## mapply( )

La función ``mapply( )`` es parecida a ``sapply( )`` pero con varias variables.


``mapply(FUN,...,SIMPLIFY = TRUE,USE.NAMES=TRUE)``

donde 

- ``...`` son los argumentos de la función ``FUN``. Pueden ser vectores, listas o arrays
- ``simplify = TRUE`` devuelve una lista o una matriz. Si ``="array"`` devuelve un array.
- ``USE.NAMES = TRUE`` hace que, si `x` es un vector de caracteres, el resultado de aplicar ``FUN`` a cada elemento de ``x`` tendrá el mismo nombre. 

In [ ]:
# Dos listas de palabras
sujetos <- list("El gato", "La niña", "El perro")
acciones <- list("salta", "corre", "ladra")

# Crear frases combinando elementos de las dos listas
frases <- mapply(function(sujeto, accion){paste(sujeto, accion)}, sujetos, acciones)

print(frases)

In [ ]:
Longitud = c(2, 3, 4, 5)
Anchura = list(3, 4, 5, 6)

# Función para calcular el área
calcular_area <- function(longitud, anchura) {
  longitud * anchura
}

# Función para calcular el perímetro
calcular_perimetro <- function(longitud, anchura) {
  2 * (longitud + anchura)
}

# Calcular el área y el perímetro para cada objeto
areas <- mapply(calcular_area, Longitud, Anchura)
perimetros <- mapply(calcular_perimetro, Longitud, Anchura)

print("Áreas:")
print(areas)
print("Perímetros:")
print(perimetros)